In [ ]:
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits.sql.base import create_sql_agent

from langchain.chains.sql_database.query import create_sql_query_chain
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

from langchain_openai import ChatOpenAI

import os
from dotenv import load_dotenv
load_dotenv()

oai_key = os.getenv("OPENAI_API_KEY")

In [7]:


db = SQLDatabase.from_uri("sqlite:///./Chinook_Sqlite.sqlite")

def display_db_info():
    print(
        f'''
        Dialect: {db.dialect}
        
        Usable tables: {db.get_usable_table_names()}
        '''
    )
    
display_db_info()


        Dialect: sqlite
        
        Usable tables: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
        


In [10]:
llm = 'gpt-3.5-turbo'
model = ChatOpenAI(model=llm, temperature=0)
toolkit = SQLDatabaseToolkit(db=db, llm=model)

sql_agent = create_sql_agent(toolkit=toolkit, llm=model, verbose=True)

In [11]:
question = 'how many artists are there?'

res = sql_agent.invoke(question)
print(res)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackI should query the Artist table to count the number of artists.
Action: sql_db_schema
Action Input: Artist
CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/I can use a COUNT query on the ArtistId column in the Artist table to find the total number of artists.
Action: sql_db_query
Action Input: SELECT COUNT(ArtistId) FROM Artist[(275,)]I now know the final answer
Final Answer: There are 275 artists.

> Finished chain.
{'input': 'how many artists are there?', 'output': 'There are 275 artists.'}
